In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.corpus import wordnet
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet as wn

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# CSV 파일에서 데이터 불러오기
apps_data = pd.read_csv('cleaned_apps_data.csv')

C:\Users\user\AppData\Local\Temp\ipykernel_26636\3248333432.py:2: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  apps_data = pd.read_csv('cleaned_apps_data.csv')


In [3]:
apps_data.columns

Index(['appId', 'title', 'url', 'score', 'price', 'free', 'category',
       'installs', 'description', 'icon'],
      dtype='object')

In [4]:
# 2. 앱 이름을 불용어 리스트로 처리
app_names = apps_data['title'].apply(lambda x: x.lower()).tolist()  # 앱 이름을 소문자로 변환하여 리스트로 수집


In [5]:
# 3. 추가적인 불용어 정의
additional_stopwords = ['pdf', 'file', 'app', 'lite', 'application']

In [6]:
# 4. 기본 불용어와 추가 불용어 결합
stop_words = list(text.ENGLISH_STOP_WORDS.union(app_names).union(additional_stopwords))


In [7]:
# 4. Sentence-BERT 모델 로드 및 문장 임베딩 생성
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(apps_data['description'].values)


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# 5. K-Means 클러스터링 수행
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [9]:
# 5. TF-IDF 기반 주요 키워드 추출
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = vectorizer.fit_transform(apps_data['description'])

def get_top_keywords(tfidf_matrix, clusters, top_n=5):
    cluster_centers = np.zeros((np.unique(clusters).size, tfidf_matrix.shape[1]))
    
    for cluster in np.unique(clusters):
        cluster_centers[cluster] = tfidf_matrix[clusters == cluster].mean(axis=0)
    
    terms = vectorizer.get_feature_names_out()
    top_keywords = []
    
    for cluster in range(cluster_centers.shape[0]):
        center = cluster_centers[cluster]
        top_indices = center.argsort()[::-1][:top_n]
        keywords = [terms[i] for i in top_indices]
        top_keywords.append(keywords)
    
    return top_keywords

In [10]:
# 6. 클러스터별 상위 5개 키워드 추출
top_keywords_per_cluster = get_top_keywords(tfidf_matrix, clusters, top_n=5)

In [11]:
# 7. 각 키워드를 임베딩하여 벡터 생성
def get_keyword_embedding(keywords):
    return model.encode(keywords)

In [12]:
# 8. Cosine Similarity를 사용하여 중심 단어 선택
def select_representative_word(keywords):
    keyword_embeddings = get_keyword_embedding(keywords)
    similarity_matrix = cosine_similarity(keyword_embeddings)

    # 각 단어의 유사도 합 계산
    similarity_sums = similarity_matrix.sum(axis=1)
    
    # 유사도 합이 가장 큰 단어를 대표 단어로 선택
    representative_idx = np.argmax(similarity_sums)
    return keywords[representative_idx]

In [13]:
# 9. 클러스터별 대표 단어 생성
def generate_category_name(keywords):
    return select_representative_word(keywords)

In [14]:
# 10. 클러스터별 대표 단어 생성 및 출력
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    print(f"Cluster {cluster_num} Name: {category_name}")


Cluster 0 Name: phone
Cluster 1 Name: chat
Cluster 2 Name: reading
Cluster 3 Name: design
Cluster 4 Name: card
